# Python in the Wild
<sup>By Matt Kravetz</sup>

Welcome to the final interactive Python tutorial!  

In our previous tutorials, we introduced some of the core concepts of the Python programming langauge - data types, functions, iteration, and collections (specifically, the <code>list</code>, the <code>tuple</code>, and the <code>dictionary</code>.  We also learned how these different tools can be strung together to build powerful tools, such as our wonderful <code>what_to_wear_v4</code> function.  

In this final tutorial, we're going to attempt to build a *real* version of the <code>what_to_wear</code> function.  This function will interact with public APIs to pull in real weather data, and make a recommendation based on future weather conditions.  It will tell us what we need to pack for an upcoming Roundtable trip by checking the weather forecast. We won't do it here, but this mini-app could easily be published as a Alexa Skill or as a Google Assistant action (both of which have excellent Python API's!). Cool, huh? 

Some of the tools used here are outside the scope of the previous tutorials, and may be a bit thin on explanation.  This is intentional!  As you continue to learn how to code, you will find that *searching* for information on code that you don't yet understand is the single best way to learn.  A simple Google search along the lines of "Python [topic you don't understand]" will almost always bring you to an excellent resource to learn more. Also, feel free to experiment!  Tear apart this function, and try running bits of it in the code cells of this notebook.

<div class="alert alert-info">
    <strong>Note:</strong> 
    <p>Feel free to interact with the notebooks as you wish - you can add cells, delete cells, and edit existing cells at any point. I highly recommend doing so, 
        as experimentation is by far the best way to learn.</p>
    <p> This notebook will self-destruct after 10 minutes of inactivity, discrarding any changes you have made.
        You can always re-launch a fresh notebook by navigating to <a>10.0.0.7:8000</a>.

</div>

<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css">
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.1/jquery.min.js"></script>

<script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js"></script>

In [221]:
# Imports are a way to pull other people's projects into our work.  
# These packages will help us retrieve parse data from the internet 
import requests
import gzip
import json


def get_city_dict():
    """  Returns the city list as a dictionary where keys are the city names and values are the city ids"""
    
    URL = "http://bulk.openweathermap.org/sample/city.list.us.json.gz"
    
    city_list_compressed = requests.get(URL).content
    city_list_bytes = gzip.decompress(city_list_compressed)
    
    # The following two lines use a technique called "comprehensions".  
    # These are very powerful tools - look them up and learn more!
    city_list_json = [json.loads(x) for x in city_list_bytes.decode().split('\n')[:-1]]
    city_dict = {record['name'].lower(): record['_id'] for record in city_list_json}
    
    return city_dict

def what_to_pack(city, t_shirt_cutoff=72, light_jacket_cutoff=60):
    """ This function tells the user what to pack for a Roundtable trip.
        
        It will make a request to the OpenWeatherMap API to retrive a forecast, then will 
    
        The user can optionally provide their own "cutoff" termperatures for t-shirts and light jackets.
        The recommendation is returned to the user as a string
    """
    
    
    city = city.lower()
    city_dict = get_city_dict()
    try:
        city_id = city_dict[city]
    except KeyError:
        raise Exception("The city you asked for isn't in the city list!")
    
    # This is our API key from OpenWeatherMap.  If you would like to create your own version of this app, 
    # register for a free account on OpenWeatherMap and obtain your API key from https://home.openweathermap.org/api_keys
    API_KEY = "6b02e1d7381bb7136d4ebd53cc5bca91"
    API_URL = "http://api.openweathermap.org/data/2.5/forecast?id=" + str(city_id) + "&APPID=" + API_KEY + "&units=metric"
    r = requests.get(API_URL)
    
    # Grab our forecast from the API response
    forecast_raw = json.loads(r.content.decode())['list']
    
    # Extract the date, temperature, and main weather forecast from our forecast json dump
    hourly_forecast = [[f['dt_txt'], f['main']['temp'], f['weather'][0]['main']] for f in forecast_raw]

    # Convert the temperatures from C to F
    for fc in hourly_forecast:
        temp_in_celcius = fc[1]
        temp_in_fahrenheit = ((temp_in_celcius * 9) / 5) + 32  
        fc[1] = temp_in_fahrenheit
    
    # Iterate through the hourly forecast, which gives data for the next 5 days.
    # Track what we need to pack based on the preferences passed in through the 
    # t_shirt_cutoff and light_jacket_cutoff arguments
    # Results are stored in a 'set', holds *unique* objects in an unsorted order
    max_temp = None
    min_temp = None
    packing_list = set()
    for time, temp, weather in hourly_forecast:
        
        # We also want to keep track of our max and min temps
        if max_temp is None or temp > max_temp:
            max_temp = temp
        if min_temp is None or temp < min_temp:
            min_temp = temp
        
        # Similar logic from our earlier examples, except we add the item to our packing_list
        if temp > t_shirt_cutoff:
            packing_list.add("t-shirt")
        elif temp > light_jacket_cutoff:
            packing_list.add("light jacket")
        else:
            packing_list.add("parka")
        
        # Also check to see if we need anything for the weather conditions
        if weather == 'Snow':
            packing_list.add("snow boots")
        elif weather == "Rain":
            packing_list.add("umbrella")
            
            # If it's cold, we should be fine in a parka or light jacket.  
            # But if it's too warm, we should bring a rain coat
            if temp > t_shirt_cutoff:
                packing_list.add("rain coat")
    
    # Build our reccomendation language.  If we hooked this into Alexa or Google Assistant, we could have 
    # a voice read this message to us!
    message = ("For your upcoming trip to {city}, you should pack the following items: {items}. The maximum temperature will be"
               " {max_temp} F.  The minimum temperature will be {min_temp} F").format(city=city.capitalize(),
                                                                                items=", ".join(packing_list),
                                                                                min_temp=min_temp,
                                                                                max_temp=max_temp)

    return message

In [223]:
what_to_pack("New York")

'For your upcoming trip to New york, you should pack the following items: umbrella, snow boots, parka. The maximum temperature will be 38.93 F.  The minimum temperature will be 23.522 F'

### Conclusion and Next Steps 

And there you have it!  This concludes our Python tutorial.  This last section went 0-60 pretty quickly.... but the intent is to demonstrate that Python is a versital tool worth digging your teeth into.  Although it is a [relatively] friendly and easy-to-learn language, it is not mearly a "beginner's langauge".  In fact, Python underpins the essential operations an uncountable number of users - from tech giants like DropBox and Google, to the quant investment powerhouses like Two Sigma and AQR, to the rapid-development cycles of Silicoln Valley start-ups, to a million side-projects from regular folks.  

It's a great skill to pick up, and I hope this tutorial inspires you to take the next steps. If you are interested in further learning, there are many, many excellent resources available on the internet. I highly reccomend Al Sweigart’s Automate the Boring Stuff with Python. Sweigart’s book offers a very soft introduction to the language, and presents it in a style that is approachable for both techincal and non-technical readers. It focuses on presenting concepts through real world examples, such as manipulating Excel and Word documents. For a more in-depth discussion, Luciano Ramalho’s Fluent Python is unparalleled.

For a video introduction to the language as a whole, I cannot reccomend the talks of Python Core Developer Raymond Hettinger highly enough. In particular, his What makes Python Awesome and Transforming Code into Beautiful, Idiomatic Python keynote talks are excellent. For a video tutorial series, YouTuber sentdex’s channel has a wealth of detailed-yet-accessible content. You can view his “Python 3 Basics Tutorial Series’ here.